In [1]:
import pandas as pd
import numpy as np
from src.reproduce_utils import Results

In [2]:
def get_result(methods, metrics, dataset_names, result_df):
    headers = ["metric", "dataset"]
    indices = [
        "method",
        "seed",
    ]
    columns = pd.MultiIndex.from_product([metrics, dataset_names], names=headers)
    index = pd.MultiIndex.from_product([methods, [1]], names=indices)
    df = pd.DataFrame(columns=columns, index=index)
    df.sort_index(inplace=True)
    for index, row in result_df.iterrows():
        for method in methods:
            if int(row["dataset_id"]) not in dataset_names:
                continue
            if "logistic" in method:
                score_method = "linear"
            else:
                score_method = method
            row_id = (method, 1)
            col = ("acc", row["dataset_id"])
            df.loc[row_id, col] = row[f"score_{score_method}"]
    return Results(df=df)

In [3]:
def get_average_rank_table(results: Results):
    datasets = results.datasets
    metrics = sorted(results.metrics, reverse=True)
    # print(results.methods)
    df = results.df
    results_rank = {}
    results_score = {}
    for metric in metrics:
        if "time" in metric:
            continue
        metric_df = df[metric]
        dataset_rank_dfs = []
        dataset_mean_dfs = []
        for dataset in datasets:
            if dataset not in metric_df.columns:
                continue
            dataset_rank_df = metric_df[dataset].groupby('method').mean().rank(ascending=False)
            dataset_rank_dfs.append(dataset_rank_df)
            dataset_mean_dfs.append(metric_df[dataset])

        results_rank[metric] = pd.concat(dataset_rank_dfs).groupby("method").mean()
        
        results_score[metric] = pd.concat(dataset_mean_dfs).groupby("method").mean()
    score_df = pd.DataFrame(results_score).reset_index()
    rank_df = pd.DataFrame(results_rank).reset_index()
    final_table = rank_df.merge(score_df, on="method", suffixes=[" Mean Rank", " Mean Score"]).T
    final_table.columns = final_table.iloc[0]
    final_table = final_table.iloc[1:]
    return final_table

def pprint(df):
    for column in df:
        df[column] = df[column].astype('float').round(decimals=4)

    print(df.to_markdown())

In [ ]:
methods = ["rf", "tree", "mlp", "hgbt", "logistic"]
metrics = ["acc"]



In [ ]:
too_easy_result_df = pd.read_csv("too_easy_without_resnet.csv").drop(["Unnamed: 0"], axis=1)
too_easy_dataset_names = too_easy_result_df["dataset_id"].astype(int).copy().to_list()
too_easy_result = get_result(methods, metrics, too_easy_dataset_names, too_easy_result_df)
too_easy_all_datasets_ranks = get_average_rank_table(too_easy_result)

selected_result_df = pd.read_csv("benchmark_without_resnet.csv").drop(["Unnamed: 0"], axis=1)
selected_dataset_names = selected_result_df["dataset_id"].astype(int).copy().to_list()
selected_result = get_result(methods, metrics, selected_dataset_names, selected_result_df)
selected_all_datasets_ranks = get_average_rank_table(selected_result)

In [ ]:
print(f"\n\n\nToo easy ({len(too_easy_dataset_names)} datasets from latest csv)")
pprint(too_easy_all_datasets_ranks)
print("\n\n\ngrinzstjan et al")
pprint(selected_all_datasets_ranks)

In [ ]:
all_result = Results(pd.concat([too_easy_result.df, selected_result.df]))
all_result_ranks = get_average_rank_table(all_result)
print(f"\nToo easy ({len(too_easy_dataset_names)} datasets from latest csv) + grinzstjan et al")
pprint(all_result_ranks)

In [ ]:
too_easy_result_df.to_csv("")

In [ ]:
df = pd.read_csv("Datasets tabular data benchmark - Feuille 12.csv")

In [ ]:
too_easy_result_df = pd.read_csv("too_easy_without_resnet.csv").drop(["Unnamed: 0"], axis=1)
too_easy_dataset_names = too_easy_result_df["dataset_id"].copy().to_list()
too_easy_result = get_result(methods, metrics, too_easy_dataset_names, too_easy_result_df)
too_easy_all_datasets_ranks = get_average_rank_table(too_easy_result)
too_easy_dids = []
for index, row in df.iterrows():
    try:
        if not pd.isnull(row["dataset_id"]) and row["Remove"] != 1 and (row["too_easy"] == 1 or args.all) and row["Redundant"] != 1:
            prefix_to_skip = ["BNG", "RandomRBF", "GTSRB", "CovPokElec", "PCam"]
            if not (np.any([row["dataset_name"].startswith(prefix) for prefix in
                            prefix_to_skip]) or "mnist" in row["dataset_name"].lower() or "image" in row[
                        "dataset_name"].lower() or "cifar" in row["dataset_name"].lower() or row["dataset_id"] == 1414):
                        too_easy_dids.append(int(row["dataset_id"]))
    except:
        continue
    

In [ ]:
len(too_easy_dids)

In [ ]:
len(too_easy_dataset_names)

In [ ]:
full_datasets_info = pd.read_csv("Datasets tabular data benchmark - numerical_classif-3.csv").fillna(0)

real_too_easy_dataset_ids = []
for index, row in full_datasets_info.iterrows():
    if not int(row["too_easy"]) and int(row["dataset_id"]) != 0:
        real_too_easy_dataset_ids.append(int(row["dataset_id"]))
        # print(row["dataset_id"])
real_too_easy_dataset_ids
full_datasets_info[~full_datasets_info["dataset_id"].isin(real_too_easy_dataset_ids)]["dataset_id"].astype(int).to_numpy()
full_datasets_info[~full_datasets_info["dataset_id"].isin(real_too_easy_dataset_ids)]["dataset_id"].astype(int).to_numpy()
my_results = pd.read_csv("csv_files/new_too_easy_without_resnet_numerical.csv").fillna(0)

full_datasets_info["score_hgbt"] = full_datasets_info["score_hbgt"].str.replace(',', '.').astype(float)
full_datasets_info["score_linear"] = full_datasets_info["score_logistic"].str.replace(',', '.').astype(float)
leo_results_required = full_datasets_info[full_datasets_info["dataset_id"].isin(real_too_easy_dataset_ids)][["score_hgbt", "score_linear"]].astype(float)
leo_results_required = leo_results_required.reindex(range(leo_results_required.shape[0]))
my_results_required = my_results[my_results["dataset_id"].isin(real_too_easy_dataset_ids)][["score_hgbt", "score_linear"]].astype(float)
my_results_required = my_results_required.reindex(range(leo_results_required.shape[0]))
(my_results_required['score_hgbt'] - 1.05 * my_results_required['score_linear'] > 0)
diff = my_results_required - leo_results_required 
leo_results_required.plot(kind="box")



diff.median(axis=0)
diff.plot(kind="box")
plt.grid()

In [ ]:
benchmark_without_resnet_numerical

In [4]:
too_easy_without_resnet = pd.read_csv("csv_files/new_too_easy_without_resnet_numerical.csv", index_col=None).fillna(0)
benchmark_without_resnet = pd.read_csv("csv_files/benchmark_without_resnet.csv", index_col=None).fillna(0)
too_easy_resnet = pd.read_csv("csv_files/new_too_easy_resnet_numerical_60_0.csv", index_col=None).fillna(0)
too_easy_resnet_2 = pd.read_csv("csv_files/new_too_easy_resnet_numerical_60_1.csv", index_col=None).fillna(0)
benchmark_numerical_resnet = pd.read_csv("csv_files/benchmark_resnet_numerical.csv", index_col=None).fillna(0)
# benchmark_categorical_r = pd.read_csv("csv_files/new_too_easy_without_resnet_numerical.csv").fillna(0)

too_easy_resnet = pd.concat([too_easy_resnet, too_easy_resnet_2])
benchmark_without_resnet_numerical = benchmark_without_resnet[benchmark_without_resnet["dataset_id"].isin(benchmark_numerical_resnet["dataset_id"].to_list())]


In [6]:
benchmark_results = benchmark_without_resnet_numerical.set_index("dataset_id").copy()
benchmark_results["score_resnet"] = benchmark_numerical_resnet.set_index("dataset_id")["score_resnet"]

too_easy_results = too_easy_without_resnet.set_index("dataset_id").copy()
too_easy_results["score_resnet"] = too_easy_resnet.set_index("dataset_id")["score_resnet"]


benchmark_ids = benchmark_numerical_resnet["dataset_id"].to_list()
too_easy_dataset_ids = too_easy_resnet["dataset_id"].to_list()

In [ ]:
methods = [ "hgbt", "logistic"] # , "resnet"] # "rf", "tree", "mlp",
metrics = ["acc"]

too_easy_result = get_result(methods, metrics, too_easy_dataset_ids, too_easy_results.reset_index())
too_easy_all_datasets_ranks = get_average_rank_table(too_easy_result)

In [ ]:
too_easy_all_datasets_ranks

In [ ]:
benchmark_result = get_result(methods, metrics, benchmark_ids, benchmark_results.reset_index())
benchmark_datasets_ranks = get_average_rank_table(benchmark_result)

In [ ]:
benchmark_datasets_ranks

In [ ]:
methods = ["hgbt", "logistic"]
metrics = ["acc"]
full_datasets_info = pd.read_csv("Datasets tabular data benchmark - numerical_classif-3.csv").fillna(0)
full_datasets_info["score_hgbt"] = full_datasets_info["score_hbgt"].str.replace(',', '.').astype(float)
full_datasets_info["score_linear"] = full_datasets_info["score_logistic"].str.replace(',', '.').astype(float)

too_easy_their_result = get_result(methods, metrics, too_easy_dataset_ids, full_datasets_info)
too_easy_their_results_ranks = get_average_rank_table(too_easy_their_result)

In [ ]:
len(too_easy_result.datasets)

In [ ]:
too_easy_their_results_ranks


In [ ]:
benchmark_their_result = get_result(methods, metrics, benchmark_ids, full_datasets_info)
benchmark_their_results_ranks = get_average_rank_table(benchmark_their_result)

In [ ]:
benchmark_their_results_ranks

In [7]:
all_my_results = pd.concat([too_easy_results, benchmark_results])

In [ ]:
all_my_results

In [8]:

all_result = get_result(methods, metrics, benchmark_ids+too_easy_dataset_ids, all_my_results.reset_index())
all_results_ranks = get_average_rank_table(all_result)

NameError: name 'methods' is not defined

In [ ]:
all_results_ranks

In [9]:
x = all_my_results["score_hgbt"] < 1.05 * all_my_results["score_linear"] 
y = all_my_results["score_resnet"] < 1.05 * all_my_results["score_linear"]
their_selection_criteria = [a and b for a, b in zip(x, y)]
too_easy_on_my_dataset_ids_their_selection_criteria = all_my_results.loc[their_selection_criteria].index.to_list()
select_on_my_dataset_ids_their = all_my_results.loc[list(map(lambda x: not x, their_selection_criteria))].index.to_list()

In [10]:
x = all_my_results["score_hgbt"] < 1.05 * all_my_results["score_tree"] 
y = all_my_results["score_resnet"] < 1.05 * all_my_results["score_tree"]
tree_selection_criteria = [a and b for a, b in zip(x, y)]
too_easy_on_my_dataset_ids_tree_selection_criteria = all_my_results.loc[tree_selection_criteria].index.to_list()
select_on_my_dataset_ids_tree = all_my_results.loc[list(map(lambda x: not x, tree_selection_criteria))].index.to_list()

In [11]:
len(select_on_my_dataset_ids_their)

46

In [12]:
methods = [ "hgbt", "logistic", "resnet", "rf", "tree", "mlp",]
metrics = ["acc"]
too_easy_on_my_result = get_result(methods, metrics, too_easy_on_my_dataset_ids_their_selection_criteria, all_my_results.reset_index())
too_easy_on_my_ranks = get_average_rank_table(too_easy_on_my_result)

In [13]:
methods = [ "hgbt", "logistic", "resnet", "rf", "tree", "mlp",]
metrics = ["acc"]
select_on_my_result = get_result(methods, metrics, select_on_my_dataset_ids_their, all_my_results.reset_index())
select_on_my_ranks = get_average_rank_table(select_on_my_result)

In [17]:
select_on_my_ranks

method,hgbt,logistic,mlp,resnet,rf,tree
acc Mean Rank,1.554348,5.01087,3.141304,3.902174,2.206522,5.184783
acc Mean Score,0.74858,0.682315,0.725965,0.616807,0.742845,0.680823


In [14]:
too_easy_on_my_ranks

method,hgbt,logistic,mlp,resnet,rf,tree
acc Mean Rank,2.155914,3.66129,2.72043,3.698925,3.134409,5.629032
acc Mean Score,0.845653,0.836374,0.841734,0.821575,0.838758,0.796502


In [15]:
methods = [ "hgbt", "logistic", "resnet", "rf", "tree", "mlp",]
metrics = ["acc"]
too_easy_on_my_result_tree = get_result(methods, metrics, too_easy_on_my_dataset_ids_tree_selection_criteria, all_my_results.reset_index())
too_easy_on_my_ranks_tree = get_average_rank_table(too_easy_on_my_result_tree)

In [16]:
methods = [ "hgbt", "logistic", "resnet", "rf", "tree", "mlp",]
metrics = ["acc"]
select_on_my_result_tree = get_result(methods, metrics, select_on_my_dataset_ids_tree, all_my_results.reset_index())
select_on_my_ranks_tree = get_average_rank_table(select_on_my_result_tree)

In [18]:
select_on_my_ranks_tree

method,hgbt,logistic,mlp,resnet,rf,tree
acc Mean Rank,1.690217,4.168478,2.918478,3.625,2.918478,5.679348
acc Mean Score,0.764519,0.728615,0.749975,0.689699,0.753759,0.688339


In [19]:
too_easy_on_my_ranks_tree

method,hgbt,logistic,mlp,resnet,rf,tree
acc Mean Rank,2.478723,3.989362,2.744681,4.042553,2.648936,5.095745
acc Mean Score,0.909462,0.896527,0.908041,0.879303,0.911267,0.895006
